In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pylab as plt
%pylab inline

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
data = pd.read_csv()

In [ ]:
data.describe() # хар-ки цифровых данных 
data["NewAge"] = data["Age"].apply(lambda x: x+100) # менять что-то в данных
data.dtypes # типы каждого столбца
data['Gender'].value_counts()
data['Gender'] = data['Gender'].map({'M' : 1, 'F' : 0})
set(data['Age'])
y = (y == 'Yes').astype(int) # как перевести Yes/No в 1/0
data.isnull().sum() # пропуски

In [ ]:
num_cols = data.columns[data.dtypes == 'int64'].tolist() # колонки только числовых данных
X = data[num_cols]

In [ ]:
# Перевод в OHE

data = pd.get_dummies(data, columns=['City_Category'], drop_first=True)

In [ ]:
y = data['Survived'] # целевая переменная (вектор ответов)

X = data.drop('Survived', axis=1) # матрица объект-признак

X.head()

### Seaborn

In [ ]:
sns.barplot(y='Purchase', x='Age', data=data)

In [ ]:
plt.figure(figsize=(12,9))

corr = data.corr()

sns.heatmap(corr,annot=True,linewidths=.5,fmt= '.2f',\
            mask=np.zeros_like(corr, dtype=np.bool), \
            cmap=sns.diverging_palette(100,200,as_cmap=True), square=True)

In [ ]:
# Удалить столбец, плохо коррелирующий с ЦП

del data['lalala']

# Добавить столбец в квадрате

data['NEW'] = data['OLD'] ** 2

In [ ]:
# Построить распределение всех признаков в отдельности с ЦП

for c in data.columns:
    if c != 'target':
        print(c)
        scatter(data[c], data['target'])
        show()

Первый подход - меняем модель на более сложную.

Подходы к улучшению качества, не меняя модель:

1) Искать мультиколлинеарность, удалять зависимые признаки. +

2) Пытаться удалить некоррелирующие с таргетом признаки. -

в линейной модели:

3) Смотрим на графики зависимости таргета с каждым признаком в отдельности, и пытаемся брать функции от признаков (квадрат, синус, экспонента...) +

4) Добавляем нелинейные взаимодействия признаков (a*b, a**b ит.д.) -

Послевательно добавляйте новые признаки, после каждого шага смотрите, как изменилось качество модели на кросс-валидации:
    
    * квадрат признака, имеющего наибольшую по модулю корреляцию с таргетом
    * произведение двух признаков, имеющих наибольшую по модулю корреляцию с таргетом

## Линейная регрессия

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2) 

model = LinearRegression()
model.fit(Xtrain, ytrain) # обучение модели


pred_test = model.predict(Xtest) # делаем предсказание
mean_squared_error(pred_test, ytest) ** 0.5 # чтобы были не рубли в квадрате

pred_train = model.predict(Xtrain) # сравниваем с train чтобы не было переобучения
mean_squared_error(pred_train, ytrain) ** 0.5


print(model.coef_) # веса при признаках (отдельные веса не должны вильно влиять на модель, те быть очень большими)
print(model.intercept_) # свободный коэффициент (w0)

### Линейная регрессия на кросс-валидации

In [ ]:
from sklearn.model_selection import cross_val_score

print(-cross_val_score(LinearRegression(), X, y, cv=3, 
                       scoring='neg_mean_squared_error').mean())

print(cross_val_score(LinearRegression(), X, y, cv=3, 
                       scoring='r2').mean())

### Линнейная регрессия с нормировкой

In [ ]:
from sklearn.preprocessing import StandardScaler

y = data['Purchase']
X = data.drop(['Purchase'],axis=1)

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=111)

scaler = StandardScaler().fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xtest = scaler.transform(Xtest)

regressor = LinearRegression()
regressor.fit(Xtrain, ytrain)

ypred_train = regressor.predict(Xtrain)
ypred_test = regressor.predict(Xtest)

r2_score(ytrain,ypred_train), r2_score(ytest,ypred_test)

#### смотреть коэффициенты

In [ ]:
coefficients = pd.concat([pd.DataFrame(X.columns),pd.DataFrame(np.transpose(regressor.coef_))], axis = 1)
print(coefficients)
plot(regressor.coef_)

## Логистическая регрессия

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
scaler.fit(X_train) # алгоритм находит параметры: m и sigma

X_train_sc = scaler.transform(X_train) # нормирует: x->(x-m)/sigma
X_test_sc = scaler.transform(X_test)


model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy_score(y_pred, y_test)

In [ ]:
from sklearn.model_selection import cross_val_score

print(cross_val_score(LogisticRegression(), X, y, cv=3, 
                       scoring='accuracy').mean())

### Логистическая регрессия пайплайн

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler


pipe = Pipeline([
    ('scaler_', StandardScaler()),
    ('model_', LogisticRegression())
    ])

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(accuracy_score(y_test, y_pred))

In [ ]:
# табличка с коэффициентами
coefs = pd.DataFrame(model.coef_, index=np.arange(len(model.coef_)), columns=X_train.columns)
coefs['Intercept'] = model.intercept_
coefs

### Кросс-валидация в пайплайне (лог регрессия)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


pipe = Pipeline([
    ('scaler_', StandardScaler()),
    ('model_', LogisticRegression())
    ])

cross_val_score(pipe, X, y, cv=3).mean()